# MEDICIONES SOBRE TIERRA

In [1]:
import numpy as np
from scipy import signal
from scipy.fftpack import fft, fftfreq, fftshift
from scipy import stats

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


import os

In [5]:
%run check_import.ipynb
%run to_baseband.ipynb
%run FFT_ordenada.ipynb
%run to_dB.ipynb
%run mv_to_ep.ipynb
%run R_Fresnel.ipynb

In [6]:
#-----------------------------------
## Parámetros globales de la antena
#-----------------------------------

Fs = 23.328e9 #Frecuencia de muestreo [Hz] 
Ts = 1/Fs
BW = 1.5e9 
corte = 150
Nfft= 1024 # nro de frecs -> conviene que sea multiplo de 2 para calcular la FFT de manera eficiente. 
#Tiene que ser mayor a la longitud de la señal (285 para 2m, 439 para 3m)

In [7]:
#------------------------------
#Elegir frecuencia portadora
#------------------------------

#portadora = 'Fc7'
portadora = 'Fc8'

if portadora == 'Fc7':
    fc = 7.290e9 
    Fc = '7.29GHz'
elif portadora == 'Fc8':
    fc = 8.748e9
    Fc = '8.74GHz'
else:
    print('Error: portadora debe ser Fc7 o Fc8')

## Mediciones

In [14]:
path_may = r'C:\Users\Coraline\Documents\UWB\tesis\mediciones_05_27'

h1_may, S1_may = check_import_win(path_may, 'S1', portadora)
#print('h =',h1_may, 'dim. S1 =',np.shape(S1_may))
S1_may_avg = np.mean(S1_may[:, 0:, :, 1:],axis=(1,3))
#print('dim. S1_avg =', np.shape(S1_may_avg))

h2_may, S2_may = check_import_win(path_may, 'S2', portadora)
#print('h =',h2_may,'dim. S2 =', np.shape(S2_may))
S2_may_avg = np.mean(S2_may[:, 0:, :, 1:],axis=(1,3))
#print('dim. S2_avg =', np.shape(S2_may_avg))

#---------------------------------------------------------------------------
path_june = r'C:\Users\Coraline\Documents\UWB\tesis\mediciones_06_09'

h1_june, S1_june = check_import_win(path_june, 'S1', portadora)
#print('h =',h1_june, 'dim. S1 =',np.shape(S1_june))
S1_june_avg = np.mean(S1_june[:, 0:, :, 1:],axis=(1,3))
#print('dim. S1_avg =', np.shape(S1_june_avg))

h2_june, S2_june = check_import_win(path_june, 'S2', portadora)
#print('h =',h2_june,'dim. S2 =', np.shape(S2_june))
S2_june_avg = np.mean(S2_june[:, 0:, :, 1:],axis=(1,3))
#print('dim. S2_avg =', np.shape(S2_june_avg))




h = [0.37  0.465 0.66  0.865] dim. S1 = (4, 10, 284, 11)
dim. S1_avg = (4, 284)
h = [0.365 0.47  0.665 0.865] dim. S2 = (4, 10, 284, 11)
dim. S2_avg = (4, 284)


dim. S1_avg = (4, 284)
